In [109]:
import pandas as pd
import numpy as np

In [110]:
# Generate simulated data for PM
pm_data = pd.DataFrame({
    'nom': np.random.choice(['PM1', 'PM2', 'PM3', 'PM4'], size=5),
    'id': np.random.randint(1000, 1005, size=5)
})

# Generate simulated data for PP (three datasets)
pp_data1 = pd.DataFrame({
    'nom': np.random.choice(['PP1', 'PP2', 'PP3'], size=5),
    'id': np.random.randint(2000, 2005, size=5)
})
pp_data2 = pd.DataFrame({
    'nom': np.random.choice(['PP4', 'PP5', 'PP6'], size=5),
    'id': np.random.randint(2005, 2010, size=5)
})
pp_data3 = pd.DataFrame({
    'nom': np.random.choice(['PP7', 'PP8', 'PP9'], size=5),
    'id': np.random.randint(2010, 2015, size=5)
})

In [111]:
pm_data.head(),pm_data.shape,pp_data1.head(),pp_data1.shape,pp_data2.head(),pp_data2.shape,pp_data3.head(),pp_data3.shape


(   nom    id
 0  PM2  1003
 1  PM3  1004
 2  PM1  1000
 3  PM4  1002
 4  PM1  1002,
 (5, 2),
    nom    id
 0  PP1  2002
 1  PP3  2001
 2  PP1  2000
 3  PP2  2001
 4  PP1  2000,
 (5, 2),
    nom    id
 0  PP4  2007
 1  PP4  2006
 2  PP4  2005
 3  PP6  2009
 4  PP4  2007,
 (5, 2),
    nom    id
 0  PP8  2014
 1  PP9  2012
 2  PP8  2010
 3  PP7  2013
 4  PP7  2012,
 (5, 2))

In [112]:
# Concatenate PP datasets
pp_concat = pd.concat([pp_data1, pp_data2, pp_data3])


In [113]:
pp_concat.head(15), pp_concat.shape

(   nom    id
 0  PP1  2002
 1  PP3  2001
 2  PP1  2000
 3  PP2  2001
 4  PP1  2000
 0  PP4  2007
 1  PP4  2006
 2  PP4  2005
 3  PP6  2009
 4  PP4  2007
 0  PP8  2014
 1  PP9  2012
 2  PP8  2010
 3  PP7  2013
 4  PP7  2012,
 (15, 2))

In [114]:
# Linking table as described
linking_data = pd.DataFrame({
    'pm': ['PM2', 'PM2', 'PM3', '', 'PM3', ''],
    'sci': ['', 'PM4', 'PM4', 'PM5', '', 'PM5'],
    'pp': ['PP1', 'PP2', 'PP3', 'PP4', 'PP5', 'PP6']
})

print(linking_data)

    pm  sci   pp
0  PM2       PP1
1  PM2  PM4  PP2
2  PM3  PM4  PP3
3       PM5  PP4
4  PM3       PP5
5       PM5  PP6


In [115]:
# Unify 'pm' and 'sci' where 'sci' is not empty
linking_data['pm'] = linking_data.apply(lambda x: x['sci'] if x['pm'] == '' else x['pm'], axis=1)
print(linking_data)

    pm  sci   pp
0  PM2       PP1
1  PM2  PM4  PP2
2  PM3  PM4  PP3
3  PM5  PM5  PP4
4  PM3       PP5
5  PM5  PM5  PP6


In [116]:
linking_data['sci'] = linking_data.apply(lambda x: '' if x['sci'] == x['pm'] else x['sci'], axis=1)
print(linking_data)

    pm  sci   pp
0  PM2       PP1
1  PM2  PM4  PP2
2  PM3  PM4  PP3
3  PM5       PP4
4  PM3       PP5
5  PM5       PP6


In [117]:
# Identify rows where 'sci' is not empty and copy them to create new rows
additional_rows = linking_data[linking_data['sci'] != ''].copy()
additional_rows.loc[:, 'pm'] = additional_rows['sci']
additional_rows.loc[:, 'sci'] = ''

# Append new rows to the original dataframe
linking_data = pd.concat([linking_data, additional_rows], ignore_index=True)

# Drop the 'sci' column if it's no longer needed
linking_data.drop('sci', axis=1, inplace=True)

# Display the updated DataFrame
print(linking_data)

    pm   pp
0  PM2  PP1
1  PM2  PP2
2  PM3  PP3
3  PM5  PP4
4  PM3  PP5
5  PM5  PP6
6  PM4  PP2
7  PM4  PP3


In [118]:
pm_data

,nom,id
0,PM2,1003
1,PM3,1004
2,PM1,1000
3,PM4,1002
4,PM1,1002


In [119]:
# first merge between pm_data and linking_data
transit_data = pm_data.merge(linking_data, left_on='nom', right_on='pm', how='inner')
transit_data

,nom,id,pm,pp
0,PM2,1003,PM2,PP1
1,PM2,1003,PM2,PP2
2,PM3,1004,PM3,PP3
3,PM3,1004,PM3,PP5
4,PM4,1002,PM4,PP2
5,PM4,1002,PM4,PP3


In [120]:
#final merge between transit_data and pp_concat
final_data = transit_data.merge(pp_concat, left_on='pp', right_on='nom', how='inner', suffixes=('_pm', '_pp'))
final_data

,nom_pm,id_pm,pm,pp,nom_pp,id_pp
0,PM2,1003,PM2,PP1,PP1,2002
1,PM2,1003,PM2,PP1,PP1,2000
2,PM2,1003,PM2,PP1,PP1,2000
3,PM2,1003,PM2,PP2,PP2,2001
4,PM4,1002,PM4,PP2,PP2,2001
5,PM3,1004,PM3,PP3,PP3,2001
6,PM4,1002,PM4,PP3,PP3,2001


In [121]:
# final cleaning: drop the columns 'nom_pm' and 'nom_pp'
final_data.drop(['nom_pm', 'nom_pp'], axis=1, inplace=True)
final_data

,id_pm,pm,pp,id_pp
0,1003,PM2,PP1,2002
1,1003,PM2,PP1,2000
2,1003,PM2,PP1,2000
3,1003,PM2,PP2,2001
4,1002,PM4,PP2,2001
5,1004,PM3,PP3,2001
6,1002,PM4,PP3,2001
